# Notebook 5: Making Decisions And Adaptive Acquisitions

Decisions are a type of object that takes in Processed Data and returns a suggested Acquisition which will be acquired.
Decisions have a propose function which takes in data and the previous acqusition and created a new one.
If a decision returns a AcquisitionPlugin type, then it will be acquired and if it returns a None type no acuisition occurs.

**DecisionNull** is a decision which always returns None (ie: does nothing)

**DecisionRepeat** is a decision which always returns the origional acquisition

**DecisionSelectOptimalZPlaneFromSharpestZ** returns an acquisition which containes the sharpect Z-Plane only. requires data that has the 'sharpest' data processed on it.

**DecisionPickROIFromXYSpotLocations** picks images which contain a certain threshhold of spots

**Decision** is a an object with some simple logic that can be created ad-Hoc

**DecisionFromCallback** is the most customizable version which enables any function to be written for making decisions.


In [2]:
import sys,os
sys.path.append('source')

  ## Example 5.1: Making A decision that repeats the previous acquisition.

  Decisions follow a similar structure to Image Processing.

In [3]:
from source.decision import *

data=[]
acq=AcquisitionPlugin()
decision=DecisionNull()
new_acq=decision.propose(data,acq)


In [4]:
decision=DecisionRepeatAcquisition()
new_acq=decision.propose(data,acq)

In [5]:
from source.postprocessors import *
data=NDTiffDataset('data/core/xyLooseGrid_5')
acq=AcquisitionPlugin()
processor=PostProcessPipeline()
processor.add('sharpest',squish_axes='z')
processedData=processor.process(data,acq)
decision=DecisionSelectOptimalZPlaneFromSharpestZ()
new_acq=decision.propose(processedData,acq)

Dataset opened                 
{frozenset({('channel', 'DAPI'), ('position', 0)}): [{'channel': 'DAPI', 'z': 0, 'position': 0}]}
{frozenset({('channel', 'DAPI'), ('position', 0)}): [{'channel': 'DAPI', 'z': 0, 'position': 0}], frozenset({('position', 0), ('channel', 'Cy5')}): [{'position': 0, 'z': 0, 'channel': 'Cy5'}]}
frozenset({('channel', 'DAPI'), ('position', 0)}) maps to {'sharpest_value': 170796866.0, 'sharpest_index': 0}
frozenset({('position', 0), ('channel', 'Cy5')}) maps to {'sharpest_value': 137480784.0, 'sharpest_index': 0}



In [9]:
decision=DecisionPickROIFromXYSpotLocations()
new_acq=decision.propose(data,acq)

NameError: name 'ProcessedData' is not defined

In [8]:
from source.decision import *

data=[]
acq=AcquisitionPlugin()

decision=DecisionNull()
new_acq=decision.propose(data,acq)

decision=DecisionRepeatAcquisition()
new_acq=decision.propose(data,acq)

#decision=DecisionIfThen().IfThreshHoldGreaterThan('mean',10).thenRemoveElement()
#decision=DecisionIfThen().IfThreshHoldLessThan('mean',10).thenChangeSetting('channelRange',['channel',['Cy5'],[100]])

In [ ]:
decision=DecisionRepeatAcquisition()
new_acq=decision.propose(data,acq)

## Example X.X: Making your own decision

DecisionFromCallback is a class that takes in a function and makes it a decision, the callback needs a particular structure to work.

In [ ]:
def function(self,processed_data,acq):
    print('hello world')
    return None

decision=DecisionFromCallback(function=function)
new_acq=decision.propose(data,acq)

In [ ]:
def helloFunction(self,processed_data,acq):
    print('hello world')
    return None

decision=Decision(function=helloFunction)
new_acq=decision.propose(data,acq)

In [ ]:
def paddingFunction(self,processed_data,acq,padding=None):
    # todo
    return None

decision=Decision(function=paddingFunction,padding=10)
new_acq=decision.propose(data,acq)

In [ ]:
def countingFunction(self,processed_data,acq,padding=None):
    # todo
    self.counter=self.counter+1
    if self.counter==5:
        return acq
    return None

decision=Decision(function=countingFunction,padding=10)
new_acq=decision.propose(data,acq)

In [ ]:
from source.apd import *
acqLibrary=AcquisitionPluginLibrary()
acquisition=acqLibrary.xySequence([[0,0]])

process=PostProcessPipeline()
process.add('mean')

decision=DecisionRepeatAcquisition()

apdSystem = APDSystem(configFileName='myConfig.cfg', rootDataFolder='') 
apdSystem.linkAPD(acquisition=acquisition,process=process,decision=DecisionRepeatAcquisition())
apdSystem.run()